# Deep Learning for NLP - Project

RULES:

* Do not create any additional cell

* Fill in the blanks

* All cells should be runnable (modulo trivial compatibility bugs that we'd fix)

* 4 / 20 points will be allocated to the clarity of your code

* Efficient code will have a bonus

DELIVERABLE:

* this notebook
* the predictions of the SST test set

DO NOT INCLUDE THE DATASETS IN THE DELIVERABLE..

In [1]:
import io
import os
import numpy as np
import scipy
import re
from nltk.tokenize import word_tokenize
from scipy import linalg
import random as rn
import string 
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
PATH_TO_DATA = os.getcwd()+"\data"

# 1) Monolingual (English) word embeddings 

In [3]:
class Word2vec():
    def __init__(self, fname, nmax=100000):
        self.load_wordvec(fname, nmax)
        self.word2id = dict.fromkeys(self.word2vec.keys())
        self.id2word = {v: k for k, v in self.word2id.items()}
        self.embeddings = np.array(list(self.word2vec.values()))
            
    def load_wordvec(self, fname, nmax):
        self.word2vec = {}        
        with io.open(fname, encoding='utf-8') as f:
            next(f)
            for i, line in enumerate(f):
                word, vec = line.split(' ', 1)
                self.word2vec[word] = np.fromstring(vec, sep=' ')
                if i == (nmax - 1):
                    break
        # New variable 
        self.lst_words = list(self.word2vec.keys())
        print('Loaded %s pretrained word vectors' % (len(self.word2vec)))
        
    def most_similar(self, w, K=5):
        # K most similar words: self.score  -  np.argsort 
        similarities = []
        pivot = self.word2vec[w]
        
        for i in range(len(self.embeddings)):
            tmp_val = self.embeddings[i]
            similarities.append((self.lst_words[i], np.dot(pivot,tmp_val)/(np.linalg.norm(pivot) * np.linalg.norm(tmp_val)) ))
            
        similarities.sort(key = lambda x : x[1], reverse = True)
        best_words = []
        
        for j in range(K):
            best_words.append(similarities[j+1][0])
            
        return best_words
    
    def KNN(self,pivot, K=5):
        
        # K most similar words: self.score  -  np.argsort 
        similarities = []
        for i in range(len(self.embeddings)):
            tmp_val = self.embeddings[i]
            similarities.append((self.lst_words[i], np.dot(pivot,tmp_val)/(np.linalg.norm(pivot) * np.linalg.norm(tmp_val)) ))
            
        similarities.sort(key = lambda x : x[1], reverse = True)
        best_words = []
        
        for j in range(K):
            best_words.append(similarities[j+1][0])
            
        return best_words
        
    def score(self, w1, w2):
        # cosine similarity: np.dot  -  np.linalg.norm
        
        # we need the vector representation of w1 and w2. The conclusion is trivial.
        # for the moment i initialize the value randomly
        
        v1 = self.word2vec[w1]
        v2 = self.word2vec[w2]

        return np.dot(v1,v2)/(np.linalg.norm(v1) * np.linalg.norm(v2))


In [4]:
w2v = Word2vec(os.path.join(PATH_TO_DATA, 'crawl-300d-200k.vec'), nmax=100000)

# You will be evaluated on the output of the following:
for w1, w2 in zip(('cat', 'dog', 'dogs', 'paris', 'germany'), ('dog', 'pet', 'cats', 'france', 'berlin')):
    print(w1, w2, w2v.score(w1, w2))
for w1 in ['cat', 'dog', 'dogs', 'paris', 'germany']:
    print(w2v.most_similar(w1))

Loaded 100000 pretrained word vectors
cat dog 0.671683666279249
dog pet 0.6842064029669219
dogs cats 0.7074389328052403
paris france 0.7775108541288561
germany berlin 0.7420295235998392
['cats', 'kitty', 'kitten', 'feline', 'kitties']
['dogs', 'puppy', 'Dog', 'doggie', 'canine']
['dog', 'pooches', 'Dogs', 'doggies', 'canines']
['france', 'Paris', 'london', 'berlin', 'tokyo']
['austria', 'europe', 'german', 'berlin', 'poland']


In [21]:
class BoV():
    def __init__(self, w2v):
        self.w2v = w2v
    
    def encode(self, sentences, idf=False):
        # takes a list of sentences, outputs a numpy array of sentence embeddings
        # see TP1 for help      
                
        sentemb = []
        #for sent in sentences:
            #print("My sentence is : {0}".format(sent))
        #    if idf is False:
                # mean of word vectors
                #assert False, 'TODO: fill in the blank'
       #         if (len(sent) > 0) :
       #             sentemb.append(np.sum([self.w2v.word2vec.get(w, [0] * 300) for w in sent],axis = 0)/len(sent))

        #    else:
                # idf-weighted mean of word vectors
                #assert False, 'TODO: fill in the blank'
        #        idf_weights = self.build_idf(sentences)
                
                #print("The idf_weights are :".format(list(idf_weights.values())))
                
                # P: how to consider the weights?
         #       if (len(sent) > 0) :
         #           sentemb.append(np.sum([ [float(idf_weights[w])*component for component in self.w2v.word2vec.get(w, [0] * 300)] for w in sent],axis = 0))
       
        if idf is False:
            for sent in sentences:
                admitted_words = []
                for w in sent:
                    try:
                        admitted_words.append(self.w2v.word2vec[w])
                    except:
                        pass
                if(len(admitted_words) == 0):
                    sentemb.append([0]*300)
                else:
                    sentemb.append(np.sum([wv for wv in admitted_words], axis = 0)/len(admitted_words))
                
        else:
            #idf_weights = self.build_idf(sentences)
            for sent in sentences:
                admitted_words = []
                for w in sent:
                    try:
                        admitted_words.append( (w, self.w2v.word2vec[w]) )
                    except:
                        pass
                if(len(admitted_words) == 0):
                    sentemb.append([0]*300)
                else:
                    cst_norm = np.sum([idf[w[0]] for w in admitted_words])
                    sentemb.append(np.sum([ [float(idf[w[0]])*component for component in w[1]] for w in admitted_words],axis = 0)/cst_norm)
                
        return np.vstack(sentemb)

    def most_similar(self, s, sentences, idf=False, K=5):
        # get most similar sentences and **print** them
        print(" I - The number of sentences in this corpus is: {}".format(len(sentences)))
        keys = self.encode(sentences, idf)
        query = self.encode([s], idf)
        
        # K most similar words: self.score  -  np.argsort 
        similarities = []
        
        print("II - I've started computing distances")
        
        for i in range(len(keys)):
            tmp_val = keys[i]
            similarities.append((sentences[i], np.dot(query,tmp_val)/(np.linalg.norm(query) * np.linalg.norm(tmp_val))))
        
        print("III - I've compute the distances and I'm starting sorting")
        
        similarities.sort(key = lambda x : x[1], reverse = True)
            
        print("IV - I've finished the sorting phase")
        
        #How to get the sentence for a given vector
        for i in range(K):
            print("The {0} (st-nd-th) phrase for similarity is: {1}".format(i,similarities[i+1][0]))
            print("Its similarity is: {}".format(similarities[i+1][1]))

        return similarities[1:K+1][0]

    def score(self, s1, s2, idf=False):
        # cosine similarity: use   np.dot  and  np.linalg.norm
        v1 = self.encode([s1],idf)
        print(s2)
        v2 = self.encode([s2],idf)
        print("The score is : {}".format(np.dot(v1,v2.T)/(np.linalg.norm(v1) * np.linalg.norm(v2))))
        
        return np.dot(v1,v2.T)/(np.linalg.norm(v1) * np.linalg.norm(v2))
    
    def build_idf(self, sentences):
        # build the idf dictionary: associate each word to its idf value
        
        # Decleare the idf dictionary
        idf = {}
        # Normalized dictionary
        n_idf = {}
        
        for sent in sentences:
             for w in set(sent): # But if in the same sentence there are more than 1 w??? --> Take the set!
                idf[w] = idf.get(w, 0) + 1 
                
        # Sure it can be optimized
        for word in list(idf.keys()):
            n_idf[word] = max(1, np.log10(len(sentences) / (idf[word])))
            
        return n_idf


In [22]:
w2v = Word2vec(os.path.join(PATH_TO_DATA, 'crawl-300d-200k.vec'), nmax=250000)
s2v = BoV(w2v)

# Load sentences in "PATH_TO_DATA/sentences.txt"
sentences = []
file_name_sentences = "sentences.txt"

f = open(file_name_sentences,'r')
data = f.read().split('\n')
exclude = set(string.punctuation)
data_cleaned = [(''.join(ch for ch in line if ch not in exclude)) for line in data]
sentences = [word_tokenize(line) for line in data_cleaned if (len(line)>0)]
#sentences = [line.split(' ') for line in data_cleaned if (len(line)>0)]
f.close()
    
    
#with open(file_name_sentences) as f:
#    text = f.read()
#sentences = re.split(r' *[\.\?!][\'"\)\]]* *', text)
#sentences = [word_tokenize(sentence) for sentence in sentences]

print("I - I'm done with the reading part")

# Build idf scores for each word
idf = {} if True else s2v.build_idf(sentences)

# You will be evaluated on the output of the following:
print('' if not sentences else sentences[10])

print("Sentence n. I (Most-similar): {}".format('' if not sentences else sentences[7]))
print("Sentence n. II (Score): {}".format('' if not sentences else sentences[13]))

print("Part EV - 1")
s2v.most_similar('' if not sentences else sentences[10], sentences)  # BoV-mean
s2v.score('' if not sentences else sentences[7], '' if not sentences else sentences[13])

print("Part EV - 2")
idf = s2v.build_idf(sentences)
s2v.most_similar('' if not sentences else sentences[10], sentences, idf)  # BoV-idf
s2v.score('' if not sentences else sentences[7], '' if not sentences else sentences[13], idf)

Loaded 200000 pretrained word vectors
I - I'm done with the reading part
['1', 'smiling', 'african', 'american', 'boy']
Sentence n. I (Most-similar): ['1', 'man', 'singing', 'and', '1', 'man', 'playing', 'a', 'saxophone', 'in', 'a', 'concert']
Sentence n. II (Score): ['10', 'people', 'venture', 'out', 'to', 'go', 'crosscountry', 'skiing']
Part EV - 1
 I - The number of sentences in this corpus is: 150736
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui


pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui


pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui


pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui


pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui


...

pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui


pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui


pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui


pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui
pui


ValueError: need at least one array to concatenate

# 2) Multilingual (English-French) word embeddings

Let's consider a bilingual dictionary of size V_a (e.g French-English).

Let's define **X** and **Y** the **French** and **English** matrices.

They contain the embeddings associated to the words in the bilingual dictionary.

We want to find a **mapping W** that will project the source word space (e.g French) to the target word space (e.g English).

Procrustes : **W\* = argmin || W.X - Y ||  s.t  W^T.W = Id**
has a closed form solution:
**W = U.V^T  where  U.Sig.V^T = SVD(Y.X^T)**

In what follows, you are asked to: 

In [ ]:
# 1 - Download and load 50k first vectors of
#     https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.en.vec
#     https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.fr.vec

# TYPE CODE HERE

# DOWNLOAD
# LOAD
n_words = 50000
w2v_en = Word2vec('wiki.en.vec',n_words)
w2v_fr = Word2vec('wiki.fr.vec',n_words)

In [ ]:
# 2 - Get words that appear in both vocabs (= identical character strings)
#     Use it to create the matrix X and Y (of aligned embeddings for these words)
words_en = set(w2v_en.lst_words)
words_fr = set(w2v_fr.lst_words)
words_common = words_en.intersection(words_fr)

n_words_common = len(words_common)
n_words_distinct = n_words - n_words_common

lst_en_words_distinct = list(words_en.difference(words_common))
lst_fr_words_distinct = list(words_fr.difference(words_common))
lst_words_common = list(words_common)

# Let's call d the dimension of the vectors
d = len(w2v_en.word2vec[lst_en_words_distinct[0]])

# Build up X and Y
X = np.zeros((d,n_words_common))
Y = np.zeros((d,n_words_common))

# Here we can use a convention: words that appear in both dictionaries are put on the top.
for i in range(n_words_common):
    X[:,i] = w2v_fr.word2vec[lst_words_common[i]]
    Y[:,i] = w2v_en.word2vec[lst_words_common[i]]
    
#for i in range(n_words_distinct):
#    X[:,n_words_common+i] = w2v_fr.word2vec[lst_fr_words_distinct[i]]
#    Y[:,n_words_common+i] = w2v_en.word2vec[lst_en_words_distinct[i]]

In [ ]:
# 3 - Solve the Procrustes using the scipy package and: scipy.linalg.svd() and get the optimal W
#     Now W*French_vector is in the same space as English_vector
U, s, Vh = linalg.svd(Y.dot(X.T))
W = U.dot(Vh)

In [ ]:
# 4 - After alignment with W, give examples of English nearest neighbors of some French words (and vice versa)
#     You will be evaluated on that part and the code above
K = 5
n_showed_words = 20

print("** Pick a french word and give me the top K translations **")
# Pick a french word and give me the top K translations
for j in range(n_showed_words):
    my_index = np.random.randint(n_words)
    my_word = w2v_fr.lst_words[my_index]
    print("French word: "+my_word)
    related_words = w2v_en.KNN(W.dot(w2v_fr.word2vec[my_word]))
    print("Top K english-related words are: ")
    for l in range(K):
        print("-> "+related_words[l])
print("** Pick a english word and give me the top K translations **")
# Pick a english word and give me the top K translations
for j in range(n_showed_words):
    my_index = np.random.randint(n_words)
    my_word = w2v_en.lst_words[my_index]
    print("English word: "+my_word)
    related_words = w2v_fr.KNN((W.T).dot(w2v_en.word2vec[my_word]))
    print("Top K french-related words are: ")
    for l in range(K):
        print("-> "+related_words[l])

If you want to dive deeper on this subject: https://github.com/facebookresearch/MUSE

# 3) Sentence classification with BoV and scikit-learn

In [ ]:
# 1 - Load train/dev/test of Stanford Sentiment TreeBank (SST)
#     (https://nlp.stanford.edu/~socherr/EMNLP2013_RNTN.pdf)
def data_pp_1(fname,case):
    f = open(fname,'r')
    data = f.read().split('\n')
    exclude = set(string.punctuation)
    data_cleaned = [ (''.join(ch for ch in line if ch not in exclude)) for line in data]
    #If we have the labels
    if(case):
        sent = [word_tokenize(line[1:]) for line in data_cleaned]
        labels = [int(line[0]) for line in data_cleaned if len(line) > 0]
        f.close()
        return sent,labels
    else:
        sent = [word_tokenize(line) for line in data_cleaned]
        f.close()
        return sent
    
file_name_train = "SST/"+"stsa.fine.train.csv"
file_name_test = "SST/"+"stsa.fine.test.X.csv"
file_name_dev = "SST/"+"stsa.fine.dev.csv"

sent_train, labels_train = data_pp_1(file_name_train,True)
sent_dev, labels_dev = data_pp_1(file_name_dev,True)
sent_test = data_pp_1(file_name_test,False)

In [ ]:
# 2 - Encode sentences with the BoV model above

w2v = Word2vec(os.path.join(PATH_TO_DATA, 'crawl-300d-200k.vec'), nmax=150000)
s2v = BoV(w2v)

sent_emb_train = s2v.encode(sent_train, s2v.build_idf(sent_train))
sent_emb_dev = s2v.encode(sent_dev, s2v.build_idf(sent_dev))
sent_emb_test = s2v.encode(sent_test, s2v.build_idf(sent_test))

In [ ]:
# 3 - Learn Logistic Regression on top of sentence embeddings using scikit-learn
#     (consider tuning the L2 regularization on the dev set)

feat_name = [ "f_"+str(i) for i in range(300)]

df_train = pd.DataFrame(sent_emb_train, columns = feat_name)
df_train["class"] = labels_train
df_dev = pd.DataFrame(sent_emb_dev, columns = feat_name)
df_dev["class"] = labels_dev
df_test = pd.DataFrame(sent_emb_test, columns = feat_name)

# Building up the model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA

X_train, Y_train = df_train[feat_name].values, df_train['class'].values

clf_LR = LogisticRegression(C = 10, max_iter=10000,solver='lbfgs',multi_class='multinomial', penalty ='l2')

clf_LR.fit(X_train, Y_train)

print("--> Train")
print("The f1_score is: "+str(f1_score(Y_train, clf_LR.predict(X_train), average='macro'))) 
print("The accuracy is: "+str((accuracy_score(Y_train, clf_LR.predict(X_train))))) 

X_dev, Y_dev = df_dev[feat_name].values, df_dev['class'].values

print("--> Test")
print("The f1_score is: "+str(f1_score(Y_dev, clf_LR.predict(X_dev), average='macro'))) 
print("The accuracy is: "+str((accuracy_score(Y_dev, clf_LR.predict(X_dev)))))


print("Same thing on the reduced space")

clf_LR_PCA = LogisticRegression(C = 10, max_iter=10000,solver='lbfgs',multi_class='multinomial', penalty ='l2')

pca = PCA(n_components = 120)
pca.fit(X_train)
X_train_PCA = pca.transform(X_train)


clf_LR_PCA.fit(X_train_PCA, Y_train)

print("--> Train")
print("The f1_score is: "+str(f1_score(Y_train, clf_LR_PCA.predict(X_train_PCA), average='macro'))) 
print("The accuracy is: "+str((accuracy_score(Y_train, clf_LR_PCA.predict(X_train_PCA))))) 

X_dev, Y_dev = df_dev[feat_name].values, df_dev['class'].values

X_dev_PCA = pca.transform(X_dev)

print("--> Test")
print("The f1_score is: "+str(f1_score(Y_dev, clf_LR_PCA.predict(X_dev_PCA), average='macro'))) 
print("The accuracy is: "+str((accuracy_score(Y_dev, clf_LR_PCA.predict(X_dev_PCA)))))


In [ ]:
# 4 - Produce 2210 predictions for the test set (in the same order). One line = one prediction (=0,1,2,3,4).
#     Attach the output file "logreg_bov_y_test_sst.txt" to your deliverable.
#     You will be evaluated on the results of the test set.

X_test = df_train[feat_name].values
pred_labels_LR = clf_LR.predict(X_test)
print(pred_labels_LR)

file_out_LR = open("logreg_bov_y_test_sst.txt","w+")
for i in range(len(pred_labels_LR)):
    file_out_LR.write(str(pred_labels_LR[i])+"\n")  
    
file_out_LR.close()

In [ ]:
# BONUS!
# 5 - Try to improve performance with another classifier
#     Attach the output file "XXX_bov_y_test_sst.txt" to your deliverable (where XXX = the name of the classifier)

from sklearn import svm

clf_SVM = svm.SVC(C = 100, kernel = "sigmoid", max_iter = 10000, decision_function_shape = "ovo", gamma = "auto")
clf_SVM.fit(X_train, Y_train)

print("--> Train")
print("The f1_score is: "+str(f1_score(Y_train, clf_SVM.predict(X_train), average='macro'))) 
print("The accuracy is: "+str((accuracy_score(Y_train, clf_SVM.predict(X_train))))) 

print("--> Test")
print("The f1_score is: "+str(f1_score(Y_dev, clf_SVM.predict(X_dev), average='macro'))) 
print("The accuracy is: "+str((accuracy_score(Y_dev, clf_SVM.predict(X_dev)))))

# 4) Sentence classification with LSTMs in Keras

## 4.1 - Preprocessing

In [ ]:
import keras
import tensorflow as tf

In [ ]:
# 1 - Load train/dev/test sets of SST
PATH_TO_DATA = os.getcwd()+"\data"

def data_pp_2(fname):
    f = open(fname,'r')
    text = f.read()
    data = text.split('\n')
    f.close()
    return data
    
file_name_train = "SST/"+"stsa.fine.train.csv"
file_name_test = "SST/"+"stsa.fine.test.X.csv"
file_name_dev = "SST/"+"stsa.fine.dev.csv"

text_train = data_pp_2(file_name_train)
text_dev = data_pp_2(file_name_dev)
text_test = data_pp_2(file_name_test)

In [ ]:
# 2 - Transform text to integers using keras.preprocessing.text.one_hot function
#     https://keras.io/preprocessing/text/
n = 16000

def text_to_2Dint(text,n):
    in_pad = []
    labels = []
    for line in (line for line in text if len(line) > 1):
        in_pad.append(keras.preprocessing.text.one_hot(line[1:], n = n*1.3,filters=string.punctuation, lower=True, split=' '))
        labels.append(int(line[0]))
    return in_pad,labels

in_pad_train,labels_train = text_to_2Dint(text_train,n)
in_pad_dev,labels_dev = text_to_2Dint(text_dev,n)

**Padding input data**

Models in Keras (and elsewhere) take batches of sentences of the same length as input. It is because Deep Learning framework have been designed to handle well Tensors, which are particularly suited for fast computation on the GPU.

Since sentences have different sizes, we "pad" them. That is, we add dummy "padding" tokens so that they all have the same length.

The input to a Keras model thus has this size : (batchsize, maxseqlen) where maxseqlen is the maximum length of a sentence in the batch.

In [ ]:
# 3 - Pad your sequences using keras.preprocessing.sequence.pad_sequences
#     https://keras.io/preprocessing/sequence/

X_train_PAD = keras.preprocessing.sequence.pad_sequences(in_pad_train, maxlen=None, dtype='int32', padding='pre', truncating='pre', value=0.0)
X_dev_PAD = keras.preprocessing.sequence.pad_sequences(in_pad_dev, maxlen=None, dtype='int32', padding='pre', truncating='pre', value=0.0)

## 4.2 - Design and train your model

In [ ]:
# 4 - Design your encoder + classifier using keras.layers
#     In Keras, Torch and other deep learning framework, we create a "container" which is the Sequential() module.
#     Then we add components to this contained : the lookuptable, the LSTM, the classifier etc.
#     All of these components are contained in the Sequential() and are trained together.


# ADAPT CODE BELOW

from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Activation
from keras.utils import np_utils

embed_dim  = 100  # word embedding dimension
nhid       = 100  # number of hidden units in the LSTM
vocab_size = n  # size of the vocabulary
n_classes  = 5

model = Sequential()
model.add(Embedding(vocab_size, embed_dim))
model.add(LSTM(nhid, dropout_W=0.2, dropout_U=0.2))
model.add(Dense(n_classes, activation='softmax'))

In [ ]:
# 5 - Define your loss/optimizer/metrics

# MODIFY CODE BELOW

loss_classif     =  'categorical_crossentropy' # find the right loss for multi-class classification
optimizer        =  'adam' # find the right optimizer
metrics_classif  =  ['accuracy']

# Observe how easy (but blackboxed) this is in Keras
model.compile(loss=loss_classif,
              optimizer=optimizer,
              metrics=metrics_classif)
print(model.summary())

In [ ]:
# 6 - Train your model and find the best hyperparameters for your dev set
#     you will be evaluated on the quality of your predictions on the test set

bs = 64
n_epochs = 9

Y_train_categorical = np_utils.to_categorical(labels_train,5)
Y_dev_categorical = np_utils.to_categorical(labels_dev,5)
history = model.fit(X_train_PAD, Y_train_categorical, batch_size=bs, nb_epoch=n_epochs, validation_data=(X_dev_PAD, Y_dev_categorical))

In [ ]:
# 7 - Generate your predictions on the test set using model.predict(x_test)
#     https://keras.io/models/model/
#     Log your predictions in a file (one line = one integer: 0,1,2,3,4)
#     Attach the output file "logreg_lstm_y_test_sst.txt" to your deliverable.

# TYPE CODE HERE

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title("Model accuracy")
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.legend(["train","dev"],loc ='upper left')

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Loss function")
plt.xlabel("epochs")
plt.ylabel("Loss")
plt.legend(["train","dev"], loc ='upper left')

#print(model.evaluate(X_train_PAD,Y_train_categorical))
print(model.evaluate(X_dev_PAD,Y_dev_categorical))
